In [1]:
pip install pandas requests tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: C:\Users\gianluca\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [6]:
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from collections import Counter

# --- 1. CONFIGURAZIONE PERCORSI E PARAMETRI ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

CSV_PATH = os.path.join(PROJECT_ROOT, "data", "raw", "product_dataset.csv")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data", "images", "products")

MAX_WORKERS = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(
        f"CSV non trovato in: {CSV_PATH}\n"
        f"Directory corrente: {os.getcwd()}\n"
        f"Assicurati che 'product_dataset.csv' sia in '{os.path.join(PROJECT_ROOT, 'data', 'raw')}'"
    )

# --- 2. HEADERS CHE SIMULANO UN BROWSER REALE ---
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
}

# --- 3. FUNZIONE DI DOWNLOAD SINGOLO ---
def download_image(row):
    asset_id = row['product_asset_id']
    url = row['product_image_url']
    
    save_path = os.path.join(OUTPUT_DIR, f"{asset_id}.jpg")
    
    if os.path.exists(save_path):
        return True, None
        
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True, None
    except Exception as e:
        return False, str(e)

# --- 4. ESECUZIONE MULTITHREADING ---
def main():
    print(f"📂 Project root: {PROJECT_ROOT}")
    print(f"Caricamento dataset da {CSV_PATH}...")
    
    df = pd.read_csv(CSV_PATH)
    df = df.dropna(subset=['product_image_url'])
    print(f"Trovate {len(df)} immagini da scaricare.")
    
    success_count = 0
    errors = []
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(download_image, row): row for _, row in df.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Download in corso"):
            success, error = future.result()
            if success:
                success_count += 1
            elif error:
                errors.append(error)
    
    print(f"\n🚀 Download completato! {success_count}/{len(df)} immagini salvate in: {OUTPUT_DIR}")
    
    # --- 5. RIEPILOGO ERRORI ---
    if errors:
        error_summary = Counter()
        for e in errors:
            if "404" in e:               error_summary["404 Not Found"] += 1
            elif "403" in e:             error_summary["403 Forbidden"] += 1
            elif "429" in e:             error_summary["429 Rate Limited"] += 1
            elif "timed out" in e.lower() or "timeout" in e.lower():
                                         error_summary["Timeout"] += 1
            elif "ConnectionError" in e: error_summary["Connection Error"] += 1
            else:                        error_summary["Altro"] += 1

        print(f"\n📊 Riepilogo errori ({len(errors)} totali):")
        for err_type, count in error_summary.most_common():
            print(f"  {err_type}: {count}")
    else:
        print("\n✅ Nessun errore!")

if __name__ == "__main__":
    main()

📂 Project root: C:\Users\gianluca\Desktop\HackUDC_2026
Caricamento dataset da C:\Users\gianluca\Desktop\HackUDC_2026\data\raw\product_dataset.csv...
Trovate 27688 immagini da scaricare.


Download in corso: 100%|█████████████████████████████████████████████████████████| 27688/27688 [15:50<00:00, 29.12it/s]


🚀 Download completato! 27688/27688 immagini salvate in: C:\Users\gianluca\Desktop\HackUDC_2026\data\images\products

✅ Nessun errore!


In [9]:
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from collections import Counter

# --- 1. CONFIGURAZIONE PERCORSI E PARAMETRI ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

CSV_PATH = os.path.join(PROJECT_ROOT, "data", "raw", "bundles_dataset.csv")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data", "images", "bundles")

MAX_WORKERS = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(
        f"CSV non trovato in: {CSV_PATH}\n"
        f"Directory corrente: {os.getcwd()}\n"
        f"Assicurati che 'bundle_dataset.csv' sia in '{os.path.join(PROJECT_ROOT, 'data', 'raw')}'"
    )

# --- 2. HEADERS CHE SIMULANO UN BROWSER REALE ---
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
}

# --- 3. FUNZIONE DI DOWNLOAD SINGOLO ---
def download_image(row):
    asset_id = row['bundle_asset_id']
    url = row['bundle_image_url']
    
    save_path = os.path.join(OUTPUT_DIR, f"{asset_id}.jpg")
    
    if os.path.exists(save_path):
        return True, None
        
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True, None
    except Exception as e:
        return False, str(e)

# --- 4. ESECUZIONE MULTITHREADING ---
def main():
    print(f"📂 Project root: {PROJECT_ROOT}")
    print(f"Caricamento dataset da {CSV_PATH}...")
    
    df = pd.read_csv(CSV_PATH)
    df = df.dropna(subset=['bundle_image_url'])
    print(f"Trovate {len(df)} immagini da scaricare.")
    
    success_count = 0
    errors = []
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(download_image, row): row for _, row in df.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Download in corso"):
            success, error = future.result()
            if success:
                success_count += 1
            elif error:
                errors.append(error)
    
    print(f"\n🚀 Download completato! {success_count}/{len(df)} immagini salvate in: {OUTPUT_DIR}")
    
    # --- 5. RIEPILOGO ERRORI ---
    if errors:
        error_summary = Counter()
        for e in errors:
            if "404" in e:               error_summary["404 Not Found"] += 1
            elif "403" in e:             error_summary["403 Forbidden"] += 1
            elif "429" in e:             error_summary["429 Rate Limited"] += 1
            elif "timed out" in e.lower() or "timeout" in e.lower():
                                         error_summary["Timeout"] += 1
            elif "ConnectionError" in e: error_summary["Connection Error"] += 1
            else:                        error_summary["Altro"] += 1

        print(f"\n📊 Riepilogo errori ({len(errors)} totali):")
        for err_type, count in error_summary.most_common():
            print(f"  {err_type}: {count}")
    else:
        print("\n✅ Nessun errore!")

if __name__ == "__main__":
    main()

📂 Project root: C:\Users\gianluca\Desktop\HackUDC_2026
Caricamento dataset da C:\Users\gianluca\Desktop\HackUDC_2026\data\raw\bundles_dataset.csv...
Trovate 2331 immagini da scaricare.


Download in corso: 100%|███████████████████████████████████████████████████████████| 2331/2331 [02:21<00:00, 16.46it/s]


🚀 Download completato! 2331/2331 immagini salvate in: C:\Users\gianluca\Desktop\HackUDC_2026\data\images\bundles

✅ Nessun errore!


## CELL FOR PEOPLE THAT HAVE ONLY CPU

In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip install transformers accelerate
!pip install opencv-python Pillow
!pip install faiss-cpu
!pip install rembg
!pip install ultralytics

## CELL FOR PEOPLE THAT HAVE SUPER POWER GPU

In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install transformers accelerate
!pip install opencv-python Pillow
!pip install faiss-gpu
!pip install rembg
!pip install ultralytics

 ## CELL OF VERIFICATION (BOTH)

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponibile: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  Nessuna GPU — gira su CPU")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🎯 Device selezionato: {DEVICE}")